# Pseudotime Spatial Trajectory Inference

In this tutorial, we are using both spatial information and gene expression profile to perform spatial trajectory inference to explore the progression of Ductal Carcinoma in situ (DCIS) - Invasive Ductal Cancer (IDC)

Source: https://www.10xgenomics.com/datasets/human-breast-cancer-block-a-section-1-1-standard-1-1-0

## 1. Preparation

We are trying to keep it focus on spatial trajectory inference then every step from reading data, processing to clustering, we will give the code here to easier for user to use.

### Read and preprocess data

In [ ]:
import stlearn as st
import pathlib as pathlib

st.settings.set_figure_params(dpi=120)

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read raw data
st.settings.datasetdir = pathlib.Path.cwd().parent / "data"
data = st.datasets.visium_sge(sample_id="V1_Breast_Cancer_Block_A_Section_1")
data = st.convert_scanpy(data)

In [ ]:
# Save raw_count
data.layers["raw_count"] = data.X
# Preprocessing
st.pp.filter_genes(data, min_cells=3)
st.pp.normalize_total(data)
st.pp.log1p(data)
# Keep raw data
data.raw = data
st.pp.scale(data)

### Clustering data

In [ ]:
# Run PCA
st.em.run_pca(data, n_comps=50, random_state=0)
# Tiling image
st.pp.tiling(data, out_path="tiling", crop_size=40)
# Using Deep Learning to extract feature
st.pp.extract_feature(data)
# Apply stSME spatial-PCA option
st.spatial.morphology.adjust(data, use_data="X_pca", radius=50, method="mean")
st.pp.neighbors(data, n_neighbors=25, use_rep='X_pca_morphology', random_state=0)
st.tl.clustering.louvain(data, random_state=0)

In [ ]:
st.pl.cluster_plot(data, use_label="louvain", image_alpha=1, size=7)

In [ ]:
st.add.annotation(data, label_list=['Fatty tissue,immune/lymphoid 1 MALAT1+',
                                    'Invasive cancer,fibrous tissue 1 CXCL14+',
                                    'Invasive cancer,fibrous tissue 2 CRISP3+',
                                    'Invasive cancer,fibrous tissue, fatty tissue',
                                    'Fatty tissue,immune/lymphoid 2 IGKC+',
                                    'Fibrous tissue',
                                    'Invasive cancer,fibrous tissue (DCIS)',
                                    'Fatty tissue, Fibrous tissue',
                                    'Invasive cancer,immune/lymphoid (IDC)',
                                    'Invasive cancer,fatty tissue 3 MUC5B+',
                                    'Fatty tissue'],
                  use_label="louvain")
st.pl.cluster_plot(data, use_label="louvain_anno", image_alpha=1, size=7)

## 2. Spatial trajectory inference

### Choosing root

3733 is the index of the spot that we chose as root. It in the DCIS cluster (6).
We recommend the root spot should be in the end/begin of a cluster in UMAP space. You can find min/max point of a cluster in UMAP as root.

In [ ]:
data.uns["iroot"] = st.spatial.trajectory.set_root(data, use_label="louvain", cluster="6", use_raw=True)
st.spatial.trajectory.pseudotime(data, eps=50, use_rep="X_pca", use_label="louvain")

### Spatial trajectory inference - global level

We run the global level of pseudo-time-space (PSTS) method to reconstruct the spatial trajectory between cluster 6 (DCIS) and 8 (lesions IDC)

In [ ]:
st.spatial.trajectory.pseudotimespace_global(data, use_label="louvain", list_clusters=["6", "8"])

In [ ]:
st.pl.cluster_plot(data, use_label="louvain", show_trajectories=True, list_clusters=["6", "8"], show_subcluster=True)

In [ ]:
st.pl.trajectory.tree_plot(data)

### Transition markers detection

Based on the spatial trajectory/tree plot, we can see 2 clades are started from sub-cluster 6 and 15.
Then we run the function to detect the highly correlated genes with the PSTS values.

In [ ]:
st.spatial.trajectory.detect_transition_markers_clades(data, clade=6, use_raw_count=False, cutoff_spearman=0.4)

In [ ]:
st.spatial.trajectory.detect_transition_markers_clades(data, clade=15, use_raw_count=False, cutoff_spearman=0.4)

In [ ]:
st.spatial.trajectory.detect_transition_markers_clades(data, clade=21, use_raw_count=False, cutoff_spearman=0.4)

In [ ]:
data.uns['clade_6'] = data.uns['clade_6'][data.uns['clade_6']['gene'].map(lambda x: "RPL" not in x)]
data.uns['clade_15'] = data.uns['clade_15'][data.uns['clade_15']['gene'].map(lambda x: "RPL" not in x)]
data.uns['clade_21'] = data.uns['clade_21'][data.uns['clade_21']['gene'].map(lambda x: "RPL" not in x)]
data.uns['clade_6'] = data.uns['clade_6'][data.uns['clade_6']['gene'].map(lambda x: "RPS" not in x)]
data.uns['clade_15'] = data.uns['clade_15'][data.uns['clade_15']['gene'].map(lambda x: "RPS" not in x)]
data.uns['clade_21'] = data.uns['clade_21'][data.uns['clade_21']['gene'].map(lambda x: "RPS" not in x)]

For the transition markers plot, genes from left side (red) are negatively correlated with the spatial trajectory and genes from right side (blue) are positively correlated with the spatial trajectory.

In [ ]:
st.pl.trajectory.transition_markers_plot(data, top_genes=30, trajectory="clade_6")

In [ ]:
st.pl.trajectory.transition_markers_plot(data, top_genes=30, trajectory="clade_15")

In [ ]:
st.pl.trajectory.transition_markers_plot(data, top_genes=30, trajectory="clade_21")

In [ ]:
st.spatial.trajectory.compare_transitions(data, trajectories=["clade_15", "clade_21"])

In [ ]:
st.pl.trajectory.DE_transition_plot(data)

We also provide a function to compare the transition markers between two clades.